In [1]:
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Text, Column, Integer, DateTime, ForeignKey, JSON, Boolean, Float, event
from geoalchemy2 import Geometry, load_spatialite
from flask_caching import Cache
from sqlalchemy import event
import os
from flask import Flask
from quickannotator.db import Project, Image, AnnotationClass, Notification, Tile, Setting
from quickannotator.db import db

In [5]:
def insert_project(app, db, name, description, is_dataset_large):
    with app.app_context():
        project = Project(name=name,
                          description=description,
                          is_dataset_large=is_dataset_large)
        db.session.add(project)
        db.session.commit()
        
        project_path = f"data/{project.id}"
        if not os.path.exists(project_path):
            os.mkdir(project_path)
        
def insert_image(app, db, project_id, name, path, height, width, embedding_coord, group_id, split):
    with app.app_context():
        image = Image(project_id=project_id,
                      name=name,
                      path=path,
                      height=height,
                      width=width,
                      embedding_coord=embedding_coord,
                      group_id=group_id,
                      split=split)
        db.session.add(image)
        db.session.commit()
        
        image_path = f"data/{image.project_id}/{image.name}"
        if not os.path.exists(image_path):
            os.mkdir(image_path)
            open(f'{image_path}/{image.name}.pdf', 'a').close()
        
def insert_annotation_class(app, db, project_id, name, color, magnification, patchsize, tilesize, dl_model_objectref):
    with app.app_context():
        annotation_class = AnnotationClass(project_id=project_id,
                                           name=name,
                                           color=color,
                                           magnification=magnification,
                                           patchsize=patchsize,
                                           tilesize=tilesize,
                                           dl_model_objectref=dl_model_objectref)
        db.session.add(annotation_class)
        db.session.commit()


In [3]:
app = Flask("app")
SearchCache = Cache(config={'CACHE_TYPE': 'SimpleCache'})
SearchCache.init_app(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quickannotator.db'
os.environ['SPATIALITE_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/mod_spatialite.so'

db.app = app
db.init_app(app)
models = [Project, Image, AnnotationClass, Notification, Tile, Setting]
with app.app_context():
    event.listen(db.engine, 'connect', load_spatialite)
    db.drop_all()
    db.metadata.create_all(bind=db.engine, tables=[item.__table__ for item in models])

In [4]:
insert_project(app, db, 
               name="test2", 
               description="test", 
               is_dataset_large=False)

insert_image(app, db, 
             project_id=1, 
             name="image", 
             path="/", 
             height=1, 
             width=1, 
             embedding_coord="POINT(1 1)", 
             group_id=1, 
             split=1)

insert_annotation_class(app, db,
                        project_id=None,
                        name="Tissue Mask",
                        color="black",
                        magnification=None,
                        patchsize=None,
                        tilesize=None,
                        dl_model_objectref=None)

